In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))



import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']


from glob import glob
from pathlib import Path
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
for fnf in glob(d + '2023-SE1-0*[0-9].log'):
    name = Path(fnf).name
    fnb = name.replace('.log','')
    print( f'"{fnb}",' )
    
    
fnbs = ['2023-SE1-003_context_change_20230322_092232']
print(fnbs)  

In [ ]:
#params['width']

In [ ]:
    
dfs = []
for fnb in fnbs:
    fn = fnb + '.log'
    fnp = fnb + '.param'

    with open(pjoin(d,fn), 'r') as f:
         l = f.readline()
         truelen = len( l.split(',') )

    r = ('trial_index, current_phase_trigger, tgti_to_show,'
       ' vis_feedback_type, trial_type, special_block_type, block_ind, '
        ' feedbackX, feedbackY, unpert_feedbackX, unpert_feedbackY,'
         ' error_distance, target_coordX, target_coordY, '
         'feedbackX_when_crossing, feedbackY_when_crossing, '
         'jax1, jax2, reward, time')
    r = r.replace(' ','')
    colnames = r.split(',')
    print('len(colnames) = ',len(colnames),'truelen = ',truelen,colnames)

    nbad = 3  # recompense strip
    #colnames = ['trial_index', 'current_phase_']
    df = pd.read_csv(pjoin(d,fn), skipfooter = nbad, on_bad_lines='warn', header=0,
                    names=colnames, encoding='latin1')
    #encoding='latin-1'
    subj = fnb.split('_')[0]    
    df['subject'] = subj
    dfs += [df]

In [ ]:
dfs_full = dfs

In [ ]:
NT = 10,40
dfs_r = []
for df in dfs:
    dfs_r += [df.query('trial_index >= @NT[0] and trial_index <@NT[1]')]
dfs = dfs_r

In [ ]:
dfs = dfs_full

In [ ]:
df_unproc = pd.concat( dfs ).reset_index()
df_unproc = df_unproc.drop(labels=['index'],axis=1)
#df_unproc

from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

dfcs = []; dfccs = []; dfcpcs = []; dfctcs = [];    
for df in dfs:
    print('---------------  Starting processing new df')                    
    dfc, dfcc, dfcpc, dfctc = addBasicInfo(df, phase2trigger, params,
                                         home_position, target_coords)


    dfcc['nonhit'] = dfcc['error_distance'] > hitr
    dfcs += [dfc]; dfccs += [dfcc]; dfcpcs += [dfcpc]; dfctcs += [dfctc];   

dfcc_all  = dfcc
dfc_all   = dfc
dfcpc_all = dfcpc
dfctc_all = dfctc

subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[0]
print(subj)
dfccos  = dfcc_all.query('subject == @subj')
dfcos   = dfc_all.query('subject == @subj')
dfcpcos = dfcpc_all.query('subject == @subj')
#dfcc = dfccos
##########################
a = dfcc_all['error_area_signed'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_area_signed_nonhit'] = a > thr

a = dfcc_all['error_area_signed_scaled_ed'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_area_signed_scaled_ed_nonhit'] = a > thr

a = dfcc_all['error_lh_ang'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_lh_ang_nonhit'] = a > thr

a = dfcc_all['error_area_signed_nn_scaled_ed'].abs()
thr = a.quantile(5e-2); print(thr)
dfcc_all['error_area_signed_nn_scaled_ed_nonhit'] = a > thr

a = dfcc_all['error_area2_signed_nn_scaled_ed'].abs()
thr = a.quantile(5e-2); print(thr)
dfcc_all['error_area2_signed_nn_scaled_ed_nonhit'] = a > thr
##############################
assert 'error' not in dfcc_all.columns
df_all = dfcc_all.drop(labels=['index', 'level_0'],axis=1)  #.copy()
#df_all['non_hit_not_adj'] = df_all['nonhit']
df_all['target_inds']     = df_all['tgti_to_show']
df_all['trials']          = df_all['trial_index']

# for err sens calc
#df_all['error']          = df_all['error_area_signed']
##############################
subjects_ = list( df_all['subject'].unique() )
print(subjects_)



In [ ]:
dfcc[['trial_index','error_area_signed_nn', 'error_area2_signed_nn']]

In [ ]:
from behav_proc import computeErrSensVersions
#envs_cur = ['stable', 'random', 'all']
envs_cur = ['all']
#tgt_inds_cur = list( dfccos['tgti_to_show'].unique() )  + [None]
tgt_inds_cur =[None]
#tgt_inds_cur = [None]  # DEBUG

block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ None ]
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'
#time_lockeds=['target', 'feedback', 'trial_end']
time_lockeds=['target']

df_all['environment'] = 1111
#df_all['feedbackX'] = 1
#df_all['feedbackY'] = 1
#%debug
coln_pairs = [('error_area_signed_scaled_ed','error_area_ofb_signed_scaled_ed',
               'error_area_signed_scaled_ed_nonhit','signed_area_scaled_ed'),
           ('error_area_signed','error_area_ofb_signed','error_area_signed_nonhit','signed_area'),
              ('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

coln_pairs = coln_pairs[0:1]

coln_pairs = [ ('error_area_signed_tln_scaled_ed','error_area_ofb_signed_tln_scaled_ed',
               'error_area_signed_scaled_ed_nonhit','signed_area_tln_scaled_ed') ]
##

coln_pairs = [ ('error_area_signed_nn_scaled_ed','error_area_ofb_signed_nn_scaled_ed',
               'error_area_signed_nn_scaled_ed_nonhit','signed_area_nn_scaled_ed') ]
coln_pairs += [('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

##
coln_pairs = [ ('error_area2_signed_nn_scaled_ed',
                None,
               'error_area2_signed_nn_scaled_ed_nonhit',
                'signed_area2_nn_scaled_ed') ]

from base2 import areaBetweenTraj, calcNormCoefSectorArea
norm_coef = calcNormCoefSectorArea(params)
def trajPair2corr(dftraj1, dftraj2):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params ) * norm_coef

dfouts = []
import sys, traceback
try:
    for coln_error,coln_correction_calc,coln_nh, calc_name  in coln_pairs:
        for time_locked in time_lockeds:
            df_all_multi_tsz, ndf2vn = computeErrSensVersions(df_all, envs_cur,block_names_cur,
                pertvals_cur,gseqcs_cur,tgt_inds_cur,
                dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
                subj_list = subjects_, error_type=error_type,
                trial_shift_sizes = [1],
                addvars=[], target_info_type = 'locs',                                        
                computation_ver = 'computeErrSens3', 
                coln_nh = coln_nh,
                time_locked = time_locked,
                coln_nh_out = coln_nh + '_shifted',
                coln_error =coln_error,                                                                              
                coln_correction_calc = coln_correction_calc,                
                trajPair2corr = trajPair2corr,
                df_fulltraj = dfc,
                verbose=1,)
            df_all_multi_tsz['error_data_for_calc'] = calc_name
            dfouts += [df_all_multi_tsz]
    df_all_multi_tsz = pd.concat(dfouts).reset_index()
    
except Exception as e:
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]

    #display(ei.tb_frame)
    psf = ei
    
    lfprev = None
    lfs = []
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame
        display(stackframe)
        lfprev = lf
        lf = stackframe.f_locals
        lfs += [lf]
        psf = psf.tb_next    
        if psf is None:
            break

    print('EXC',e)

if coln_correction_calc is not None:
    df_all_multi_tsz['coln_corrc'] = df_all_multi_tsz[coln_correction_calc]
df_all_multi_tsz['coln_error'] = df_all_multi_tsz[coln_error]

bigEStis = df_all_multi_tsz.query('err_sens > 10')['trial_index'].to_numpy()
display(bigEStis)
bigEStis_neg = df_all_multi_tsz.query('err_sens < -10')['trial_index'].to_numpy()
#display(bigEStis_neg)

In [ ]:
from behav_proc import truncateDf

In [ ]:
#truncateDf?

In [ ]:
dftmp = df_all_multi_tsz.query('error_data_for_calc == @calc_name'
                ' and time_locked == "target"')
escol = dftmp['err_sens']
escolg = escol[~(np.isinf(escol) | escol.isna()) ]
escolg2 = truncateDf(dftmp[['subject','trials','err_sens',
            'trial_shift_size','trial_group_col_calc']], 
                     'err_sens', infnan_handling='discard')
np.sum( escol.isna() ), np.sum( np.isinf(escol) ), 

display( escolg.describe() )
display( escolg2['err_sens'].describe() )

In [ ]:
escolg2['err_sens'].hist(bins=20)

In [ ]:
df_all_multi_tsz['err_sens']

In [ ]:
bigEStis

In [ ]:
target_coords

In [ ]:
cols = ['err_sens', 'correction', coln_error]
df_all_multi_tsz.set_index('trial_index').loc[bigEStis[10:],cols]

In [ ]:
cols = ['err_sens', 'correction', coln_error]
df_all_multi_tsz.set_index('trial_index').loc[bigEStis[10:],cols]

In [ ]:
trial_inds_glob1, trial_inds_glob2 = lf['trial_inds_glob1'], lf['trial_inds_glob2']
errors0 = lf['errors0']

In [ ]:
df_fulltraj = dfc_all

In [ ]:
def trajPair2corr_plot(dftraj1, dftraj2, ax):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params, ax=ax ) * norm_coef

In [ ]:
from config2 import path_fig

In [ ]:
trial_inds_glob1

In [ ]:
lf['colname_nh']

In [ ]:
a = list(zip(trial_inds_glob1, \
                          trial_inds_glob2))
rng = slice(103-5,1+nshow)
a[rng]

In [ ]:
for gti1,gti2 in list(zip(trial_inds_glob1, \
                          trial_inds_glob2))[rng]:
    print(gti1)

In [ ]:
ti_ - (nshow//2)

In [ ]:
#ax = plt.gca()
correction = []


show_dist_guides = False
ww = 5; hh = 5

#nshow = 25\
nshow = 25
rng = slice(1,1+nshow)
#ti_ = 103; 
ti_ = 450; 
rng = slice(ti_ - (nshow//2), ti_ + (nshow//2) )

L = min(nshow, len(trial_inds_glob1) -2 )
nc = int(np.ceil(np.sqrt(L) ) ); nr = int( np.ceil( L / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi = 0
#for trial_inds_glob1, valid_inds1, trial_inds_glob2
for gti1,gti2 in list(zip(trial_inds_glob1, \
                          trial_inds_glob2))[rng]:
    
    ax = axsf[axi]
    
    dftraj1 = df_fulltraj.query( 'trial_index == @gti1' )
    dftraj2 = df_fulltraj.query( 'trial_index == @gti2' )

    dftraj1 = dftraj1.loc[dftraj1.index[1: ] ]
    dftraj2 = dftraj2.loc[dftraj2.index[1: ] ]

    ab = trajPair2corr_plot(dftraj1, dftraj2, ax=ax  )

    correction_cur = ab
    correction += [correction_cur]
    es = correction_cur/errors0[gti1]
    s = ''
    if es > 10:
        s = '*'
    print(s, gti1,gti2,correction_cur,
          errors0[gti1], es )
    axi += 1

plt.tight_layout()
plt.savefig(pjoin(path_fig, f'doubletraj_2D_{subj}_{rng.start,rng.stop}.pdf') )

# when first is on the right of second, then we have positive
# but to be similar to Romain's version we would like to have neg
# or not?

In [ ]:
lfs[-3]['gti1'], lfs[-3]['gti2']

In [ ]:
lfs[-3].keys()

In [ ]:
lfprev['dftraj2']

In [ ]:
dfcc.columns

# observe table

In [ ]:

lf = psf.tb_next.tb_frame.f_locals

In [ ]:
#tis = np.arange(450,467)
tis = np.arange(460,464)

In [ ]:
len(df_all_multi_tsz)

In [ ]:
print(coln_correction_calc)

In [ ]:
df_all_multi_tsz.columns

In [ ]:

cols = ['time_locked','err_sens','correction','nonhit',
 'perturbation','trial_type','prev_trial_type','coln_corrc','coln_error', 'prev_error_like',
       'trial_inds_glob_prev_error']

In [ ]:
# observe shifts
cols = ['err_sens','time_locked','trial_inds_glob_prevlike_error','trial_inds_glob_nextlike_error','coln_error', 'prev_error']
df_all_multi_tsz.query('trial_index in @tis').\
    set_index('trial_index')[cols]

In [ ]:
cols = ['err_sens','coln_error', 'prev_error_like', 'correction']
df_all_multi_tsz.query('trial_index in @tis and time_locked == "target"').\
    set_index('trial_index')[cols]

In [ ]:
#tis = np.arange(450,467)
tis = np.arange(460,464)
tis = np.arange(50,60); 55
#tis = np.arange(92,100); 96

In [ ]:
# add col prev-trial type

In [ ]:
# prev_error_like contains rubbish
# there are no shifting columns

In [ ]:
df_all_multi_tsz.query('prev_trial_type == "pause"').\
    set_index('trial_index')[cols]
#query('trial_index in @tis')[['trial_index'] + cols].set_index('trial_index')

In [ ]:
tl = 'trial_end'
df_all_multi_tsz.query('trial_index in @tis and time_locked == @tl').\
    set_index('trial_index')[cols]
#query('trial_index in @tis')[['trial_index'] + cols].set_index('trial_index')

In [ ]:
# todo:  why is corretino negative on trial_index 55?

In [ ]:
cols = ['error_data_for_calc', 'err_sens','correction','coln_corrc','coln_error', 'prev_error',
        'nonhit',  'perturbation','trial_type','prev_trial_type',
        ]
df_all_multi_tsz.query('trial_index in @tis  and time_locked == @tl').\
    set_index('trial_index')[cols]
#query('trial_index in @tis')[['trial_index'] + cols].set_index('trial_index')

In [ ]:
df_all_multi_tsz.query('err_sens < -50 and time_locked == "feedback"')[['trial_index','trialwb','err_sens','special_block_type','trial_type']]

In [ ]:
tl = 'target'
df_all_multi_tsz.query('err_sens < -15 and time_locked == @tl and special_block_type == "None"')\
    [['trial_index','trialwb','err_sens','special_block_type','trial_type']]

In [ ]:
dftmp['err_sens']

In [ ]:
dftmp['err_sens'].describe()

In [ ]:
dftmp.query('abs(err_sens) < 10')['err_sens'].hist(bins=30)

In [ ]:
dftmp['err_sens'].describe()

In [ ]:
dftmp = df_all_multi_tsz.query('time_locked == @tl')#[['trial_index','trialwb','err_sens','special_block_type','trial_type']]
dftmp = dftmp[~ dftmp['err_sens'].isna()]
dftmp = dftmp[~ np.isinf(dftmp['err_sens'] )]
ax = sns.violinplot(dftmp,y='err_sens')
#ax.set_ylim(-10500,10500)

In [ ]:
# very large negative err_sens are in error_clamp_sandwich or error_clamp_pair

In [ ]:
#except trial index 55

In [ ]:
cols = ['trialwb','err_sens',
 'perturbation',
 'trial_type',
 'error_endpoint_ang',
 'error_area_signed_nn',
 'error_area_signed',
 'time_mvt',
 'error_area_signed_tln',
 'error_area_signed_tln_scaled_ed',
 'error_lh_ang',
 'correction',
 'nonhit']

In [ ]:
cols = sum(colscols, start=[])

In [ ]:
colscols = [['trialwb','perturbation','trial_type'],
            ['error_endpoint_ang', 'error_area_signed_nn'],
            ['error_area_signed', 'time_mvt'],
            [ 'error_area_signed_tln', 'error_area_signed_tln_scaled_ed', 'error_lh_ang'],
            [ 'correction', 'prev_error'],
           ['err_sens', 'nonhit']]

# new ver

In [ ]:
colname_nh= lf['colname_nh']

In [ ]:
colname_nh

In [ ]:
sum(df_esv[colname_nh])

In [ ]:
df_esv[colname_nh]

In [ ]:
c = df_esv[colname_nh]
df_esv.loc[c,err_sens_coln]  = (correction / errors_cur)[c]


In [ ]:
df_esv = lf['df_esv']
err_sens_coln, correction =  lf['err_sens_coln'], lf['correction']
behav_df, df_inds = lf['behav_df'], lf['df_inds']
coln_error = lf['coln_error']
coln_correction_calc = lf['coln_correction_calc']; 

trial_inds_loc_cur = lf['trial_inds_loc_cur']
valid_inds_cur = lf['valid_inds_cur']
trial_inds_loc_next = lf['trial_inds_loc_next']
valid_inds_next = lf['valid_inds_next']
vals_for_corr = lf['vals_for_corr']
vals_for_corr_cur = lf['vals_for_corr_cur']
vals_for_corr_next = lf['vals_for_corr_next']
print(coln_correction_calc, err_sens_coln)

display(df_esv.loc[450:455])

In [ ]:
# from error_sensitivity import shiftVals
# vals_for_corr_next = shiftVals(vals_for_corr,
#                                trial_inds_loc_next, valid_inds_next)
# vals_for_corr_next

In [ ]:
trial_inds_loc_cur[:5], trial_inds_loc_next[:5]

In [ ]:
np.max(np.abs(trial_inds_loc_cur[1:] - trial_inds_loc_next[:-1]))

In [ ]:
valid_inds_cur[:5],valid_inds_next[:5]

In [ ]:
valid_inds_cur[-5:],valid_inds_next[-5:]

In [ ]:
#vals_for_corr_next - vals_for_corr

In [ ]:
correction

In [ ]:
vals_for_corr_cur[:5], vals_for_corr_next[:5]

In [ ]:
#lf['valid_inds_cur']

In [ ]:
(correction / errors_cur)[450:455]

# 2d plots

In [ ]:
s = subjects[0]

In [ ]:
grp_perti = dfcos.groupby(['trial_index'])

pause_inds = dfcpc_all.query(f'subject == "{s}"').groupby('trial_index').max('time').index
#['trial_index']
pause_inds = np.array(pause_inds)

#rngs = f'{rng.start,rng.stop}'
#rngs = f'S={S}; N={N}'

In [ ]:
#55,96

In [ ]:
df_all.query( 'perturbation > 0' )[['trial_index','trialwb']]

In [ ]:
#rng = np.arange(460,464);
#rng = np.arange(50,60)

rng = np.arange(50,80)
#rng = np.arange(57,58)
rng = np.arange(13,33)
rngs = f'{rng[0]}-{rng[-1]}'; print(rngs)

N = len(rng); print(N)

In [ ]:
# add to colscols vars needed to compute err sens
coln_correction_calc, coln_error

In [ ]:
from behav_proc import row2multierr
row = axi2d[1]['row']
r = row2multierr(row,dfcos,grp_perti, home_position, target_coords,
           params)
z = zip( ['err', 'error_lh_ang', 'error_unpert_lh_ang', 'start_dist',
              'traja', 'trajoa', 'td',
        'length',  'enddist'], r)
dict(z)

In [ ]:
edfc = df_all_multi_tsz['error_data_for_calc'].unique()
print(edfc)

In [ ]:
#coln_correction_calc
edfc

In [ ]:
edfc = df_all_multi_tsz['error_data_for_calc'].unique()[0]

In [ ]:
tl = 'trial_end'

In [ ]:
print(tl, edfc)
dfesqs = (f'subject == "{s}"'
          ' and error_data_for_calc == @edfc'
          ' and trial_shift_size == 1 and trial_group_col_calc == "trials"'
          ' and time_locked == @tl')    
df_all_multi_tsz.query(dfesqs)

In [ ]:
#dfes.columns

In [ ]:
from exper_protocol.utils import screen2homec
target_coords_homec = screen2homec( *tuple(zip(*target_coords)), home_position  )

In [ ]:
# angle is not good between segment from beg to border 
# and from border to target
# if I start not from target then I have to modify area computation

# I can just rotate everything aground starting point
# target coord will also change (important when I call area() )

# coords of the polygon will be  (tadja means adjusted target aligned)
#   pt of exit of fb (tadja)
#   traj (tadja)
#   tgt (tadja)

In [ ]:
df_all_multi_tsz

In [ ]:
#dfcc.columns

In [ ]:
colscols = [['trialwb','perturbation','trial_type'],
            ['prev_trial_type'],
            ['error_endpoint_ang', 'error_lh_ang'],            
            ['error_distance','time_mvt', 'traj_length'],
            ['error_area_signed_nn'],
            ['error_area_ofb_signed_nn'],           
            [coln_correction_calc], [coln_error],
            [ 'correction', 'prev_error'],
           ['err_sens','prev_err_sens', 'nonhit']]

In [ ]:
#ti = 50 # strange in the upper part
ti = 58 # strange in the upper part
df0 = dfcc.query(f'subject == "{s}" and trial_index == {ti}')
row = df0.to_dict('records')[0]

ev = row['error_endpoint_ang']
tiwb = row['trialwb']

dfes = df_all_multi_tsz.query(dfesqs + f' and trial_index == {ti}')
assert len(dfes) == 1, len(dfes)
row2 = dfes.to_dict('records')[0]
row.update(row2)

ax = plt.gca()
r = row2multierr(row, dfc, grp_perti, home_position,             
            target_coords, params, ax=ax,
            revert_pert = False, xlim=(-160,160),
            titlecols = colscols, addinfo=row)
z = zip( ['err', 'error_lh_ang', 'error_unpert_lh_ang', 'start_dist',
              'traja', 'trajoa', 'td',
        'length',  'enddist'], r)
dict(z)

In [ ]:
N  = len(rng) //2; N

In [ ]:
from behav_proc import plotTraj, plotTraj2
from config2 import path_fig
#rng = range(600,603)
#rng = range(596,625)
#rng = range(8,10)
#rng = range(50,60)
#rng = range(55,57)

show_dist_guides = False
ww = 7; hh =10
nc = int(np.ceil(np.sqrt(len(rng)) ) ); nr = int( np.ceil( len(rng) / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi2d = {}
for axi,ti in enumerate(rng):
    ax = axsf[axi]    
    if ti not in grp_perti.groups:
        if ti in pause_inds:            
            print(f'Skipping {ti} because pause')
            ax.set_title('PAUSE')
        else:
            print(f'Skipping {ti} because ?')
        continue
        
    df0 = dfcc.query(f'subject == "{s}" and trial_index == {ti}')
    row = df0.to_dict('records')[0]
    
    ev = row['error_endpoint_ang']
    tiwb = row['trialwb']
    
    dfes = df_all_multi_tsz.query(dfesqs + f' and trial_index == {ti}')
    assert len(dfes) == 1
    row2 = dfes.to_dict('records')[0]
    row.update(row2)
    
    
    idx = grp_perti.groups[ti]
    dfcurtr = dfcos.loc[idx[1:]]  # zeroth is equal to zero
    import sys, traceback
    try:
#         plotTraj2(ax, dfcurtr, home_position, target_coords_homec, params,
#                 force_entire_traj=0, calc_area = 1, addinfo=row,
#                 titlecols = colscols, xlim=(-160,160))
        
        r = row2multierr(row, dfc, grp_perti, home_position,             
            target_coords, params, ax=ax,
            revert_pert = False, xlim=(-160,160),
            titlecols = colscols, addinfo=row)
        z = zip( ['err', 'error_lh_ang', 'error_unpert_lh_ang', 'start_dist',
                      'traja', 'trajoa', 'td',
                'length',  'enddist'], r)
        dd = dict(z)
    
    except Exception as e:
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)

        psf = exc_info[2].tb_next
        stackframe = exc_info[2].tb_next.tb_frame

        display(psf.tb_frame)
        #display(psf.tb_next.tb_frame)

        local_vars_in_fun = stackframe.f_locals
        plt.close()
        break
    
    from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.set_yticks(np.arange(-50,400,50))
    #ax.xaxis.set_minor_locator(AutoMinorLocator(25))
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    #ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    axi2d[axi] = {'df0':df0, 'row':row, 'dfcurtr':dfcurtr}
    
    print(f'error = {ev}, tiwb = {tiwb}')
    
plt.tight_layout()
plt.savefig(pjoin(path_fig, f'traj_2D_{row["subject"]}_{rngs}.pdf') )
plt.close()

In [ ]:
display(psf.tb_frame)
lf = psf.tb_frame.f_locals
if psf.tb_next is not None:
    psf2 = psf.tb_next
    display(psf2.tb_frame)
    lf0 = lf
    lf = psf2.tb_frame.f_locals

#display(exc_info[2].tb_frame)

In [ ]:
lf0['fbXhc_ta'].shape

In [ ]:
lf['last_chi'], lf['chis']

In [ ]:
lf['tgtcur'] , lf['fb0pt'], lf['fbXYhc'].shape

In [ ]:
np.math.atan2(tuple( lf['tgtcur_adj'] ))

# old ver

In [ ]:
df_esv = lf['df_esv']
err_sens_coln, correction, errors =  lf['err_sens_coln'], lf['correction'], lf['errors']
behav_df, df_inds = lf['behav_df'], lf['df_inds']
coln_error = lf['coln_error']
coln_correction_calc = lf['coln_correction_calc']; 
print(coln_correction_calc, err_sens_coln)

In [ ]:
dftmp = behav_df.query('trial_index in [@tibad0, @tibad]')[['trial_index',coln_error,coln_correction_calc]]
display(dftmp)
inds = list(dftmp.index); print(inds)

In [ ]:
lf['time_locked']
vals_for_corr, vals_for_corr_next = lf['vals_for_corr'], lf['vals_for_corr_next']

In [ ]:
len(df_inds), len(df_esv)

In [ ]:
errors_cur = lf['errors_cur']

In [ ]:
lf['shiftsz']

In [ ]:
errors_cur[inds

In [ ]:
'vals_for_corr', vals_for_corr[inds], 'vals_for_corr_next', vals_for_corr_next[inds]

In [ ]:
df_esv.loc[inds,['error', 'prev_error', 'correction','err_sens']]

In [ ]:
-10.017254 / -9.986173e+00

In [ ]:
pd.DataFrame({'a':[1,2], 'b':[0.4, 3.6]})

In [ ]:
-10.017254 / 0.493040

In [ ]:
df_all_multi_tsz['time_locked']

In [ ]:
# if I timelock to target then  we take area of feedback that happened last time and 
# compare it with difference between are of
#   orig feedback this time and last time
# since my algo expects vertical path to target

In [ ]:
df_esv.loc[ range(450,457) ,['error', 'prev_error', 'correction','err_sens']]

In [ ]:
correction

In [ ]:
tibad0 = 464
tibad = 465

In [ ]:
row = df_all_multi_tsz.query('trials == 465').to_dict('records')[0]

In [ ]:
colscols = [['trialwb','perturbation','trial_type'],
            ['error_endpoint_ang', 'error_area_signed_nn'],
            ['error_area_signed', 'time_mvt'],
            [ 'error_area_signed_tln', 'error_area_signed_tln_scaled_ed', 'error_lh_ang'],
            [ 'correction', 'prev_error'],
           ['err_sens', 'nonhit']]

# correction is different

for cols in colscols:
    for col in cols:
        print(col,row[col])

In [ ]:
row